### Homework 3

Використовуючи файл `population_prospects.csv`, створіть карти, котрі відповідають на наступні питання:

- В яких країнах світу населення у 2100 році буде більшим, ніж було у 2000 році, а в яких меншим?

- На скільки відсотків збільшиться або зменшиться населення кожної країни у 2100 році у порівнянні з 2000 роком?

- Яким буде населення країн світу у 2100 році?

У наборі даних ви знайдете декілька прогнозів щодо зростання / скорочення населення країн світу (змінна `variant`). Для фактичних історичних даних (2000 рік) ви можете використовувати значення `Estimates`, для прогнозних даних (2100 рік) ви можете використовувати значення `Medium variant`.

Ви можете використовувати файл `world-countries.json` або будь-який інший файл з геоданими, котрий вам підходить для цієї задачі. Ви можете використовувати будь-який доступний вам інструмент візуалізації даних.

Ви маєте також у невеличкому супровідному тексті обґрунтувати вибір типу карти / типу кольорової шкали, пояснити його сильні та слабкі сторони для поставленої задачі.  

In [172]:
# !pip3 install geopandas
import altair    as alt
import pandas    as pd
import geopandas as gpd

alt.data_transformers.enable('csv')
alt.renderers.enable('default')

RendererRegistry.enable('default')

### 1. Data exploration

In [173]:
geo = gpd.read_file('./world-countries.json')
geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   id        177 non-null    object  
 1   name      177 non-null    object  
 2   geometry  177 non-null    geometry
dtypes: geometry(1), object(2)
memory usage: 4.3+ KB


In [174]:
geo.head(10)

,id,name,geometry
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."
5,ARM,Armenia,"POLYGON ((43.58275 41.09214, 44.97248 41.24813..."
6,ATA,Antarctica,"MULTIPOLYGON (((-59.57209 -80.04018, -59.86585..."
7,ATF,French Southern and Antarctic Lands,"POLYGON ((68.93500 -48.62500, 69.58000 -48.940..."
8,AUS,Australia,"MULTIPOLYGON (((145.39798 -40.79255, 146.36412..."
9,AUT,Austria,"POLYGON ((16.97967 48.12350, 16.90375 47.71487..."


In [175]:
df = pd.read_csv('./population_prospects.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188000 entries, 0 to 187999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   variant       188000 non-null  object 
 1   country       188000 non-null  object 
 2   country_code  188000 non-null  int64  
 3   parent_code   188000 non-null  int64  
 4   year          188000 non-null  int64  
 5   population    188000 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 8.6+ MB


In [176]:
df.head(10)

,variant,country,country_code,parent_code,year,population
0,Estimates,Burundi,108,910,1950,2308.927
1,Estimates,Comoros,174,910,1950,159.459
2,Estimates,Djibouti,262,910,1950,62.000
3,Estimates,Eritrea,232,910,1950,822.347
4,Estimates,Ethiopia,231,910,1950,18128.030
5,Estimates,Kenya,404,910,1950,6076.757
6,Estimates,Madagascar,450,910,1950,4083.548
7,Estimates,Malawi,454,910,1950,2953.871
8,Estimates,Mauritius,480,910,1950,493.258
9,Estimates,Mayotte,175,910,1950,15.138


In [177]:
df['variant'].unique()

array(['Estimates', 'Medium variant', 'High variant', 'Low variant',
       'Constant-fertility', 'Instant-replacement', 'Momentum',
       'Zero-migration', 'Constant-mortality', 'No change'], dtype=object)

### 2. Data preprocessing

In [178]:
df = df.pivot_table(index=['country'], 
            columns=['variant'], values='population')
df = df.reset_index()

In [179]:
df = df.rename_axis(None, axis=1)
df = df.rename(columns = {'country': 'Country'}, inplace = False)

In [180]:
def uniform_names(name):
    name = name.replace("Bahamas", "The Bahamas")
    name = name.replace("Bolivia (Plurinational State of)", "Bolivia")
    name = name.replace("Iran (Islamic Republic of)", "Iran")
    name = name.replace("Congo", "Republic of the Congo")
    name = name.replace("Brunei Darussalam", "Brunei")
    name = name.replace("Czechia", "Czech Republic")
    name = name.replace("Cote d'Ivoire", "Ivory Coast")
    name = name.replace("Falkland Islands (Malvinas)", "Falkland Islands")
    name = name.replace("French Guiana", "Equatorial Guinea")
    name = name.replace("Viet Nam", "Vietnam")
    name = name.replace("Venezuela (Bolivarian Republic of)", "Venezuela")
    name = name.replace("North Macedonia", "Macedonia")
    name = name.replace("Russian Federation", "Russia")
    name = name.replace("Serbia", "Republic of Serbia")
    name = name.replace("Syrian Arab Republic", "Syria")
    name = name.replace("Lao People's Democratic Republic", "Laos")
    name = name.replace("Republic of Korea", "South Korea")
    name = name.replace("Republic of Korea", "North Korea")
    name = name.replace("Republic of Moldova", "Moldova")
    name = name.replace("China, Taiwan Province of China", "Taiwan")
    name = name.replace("Timor-Leste", "East Timor")
    
    return name

df['Country'] = df['Country'].apply(uniform_names)

In [181]:
df_with_geo = pd.merge(gpd.GeoDataFrame(geo), df, left_on="name", right_on="Country", how="left")

In [182]:
df_with_geo = df_with_geo.dropna()

In [183]:
df_with_geo.drop("name", axis='columns', inplace=True)

In [184]:
df_with_geo = df_with_geo.rename(columns = {'id': 'Country Abreviation'}, inplace = False)

In [185]:
df_with_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 168 entries, 0 to 177
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Country Abreviation  168 non-null    object  
 1   geometry             168 non-null    geometry
 2   Country              168 non-null    object  
 3   Constant-fertility   168 non-null    float64 
 4   Constant-mortality   168 non-null    float64 
 5   Estimates            168 non-null    float64 
 6   High variant         168 non-null    float64 
 7   Instant-replacement  168 non-null    float64 
 8   Low variant          168 non-null    float64 
 9   Medium variant       168 non-null    float64 
 10  Momentum             168 non-null    float64 
 11  No change            168 non-null    float64 
 12  Zero-migration       168 non-null    float64 
dtypes: float64(10), geometry(1), object(2)
memory usage: 18.4+ KB


In [186]:
df_with_geo.head()

,Country Abreviation,geometry,Country,Constant-fertility,Constant-mortality,Estimates,High variant,Instant-replacement,Low variant,Medium variant,Momentum,No change,Zero-migration
0,AFG,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",Afghanistan,138543.698988,59422.740420,17226.780155,78233.819864,55956.766309,55212.625951,65759.578407,57016.887642,122714.591086,69966.875148
1,AGO,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6...",Angola,177168.726420,89786.879160,12845.058000,120095.744605,50536.128198,86383.548284,102061.191272,47848.008667,152812.902654,102126.769370
2,ALB,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",Albania,2201.732901,1990.069444,2543.362521,2484.922852,2619.153568,1831.403667,2122.920025,3000.972296,2067.334704,2584.174074
3,ARE,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",United Arab Emirates,11123.330210,10693.688259,2760.642676,12906.112963,13718.729617,9499.602420,11091.756802,10203.820395,10724.123642,9014.107160
4,ARG,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",Argentina,60241.864543,51660.746235,30548.437394,64387.946901,55942.889988,46531.012864,54652.273469,53107.685222,57118.623370,54538.942272


### 3. Plotting

#### Task 1. В яких країнах світу населення у 2100 році буде більшим, ніж було у 2000 році, а в яких меншим?

#### Task 2. На скільки відсотків збільшиться або зменшиться населення кожної країни у 2100 році у порівнянні з 2000 роком?

#### Task 3. Яким буде населення країн світу у 2100 році?

Підказки

https://github.com/unexpectedjourney/data-visualization/blob/main/assignment-4/main.ipynb